In [21]:
from src.app.drivers import BaseDriver, OrdersPageDriver
import os
import time
import argparse
import pandas as pd
from datetime import datetime
import logging
from src.utils.log_config import setup_logger
from src.utils.data_handler import get_prettified_and_mapped_orders, convert_flattened_orders_to_df, json_str_to_file
from src.utils.data_merger import DataMerger
from src.utils.excel_formatter import ExcelFormatter


In [22]:
raw_orders = ['Order: 2ABD90DB\nDelivered\nThe order was delivered at 11:57 PM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nChenoa M\nDasher\nBrianne\nPicked Up\n11:48 PM\n(Quoted: 11:39 PM)\nSeptember 26, 2023\nDropped Off\n11:57 PM\n(Quoted: 12:00 AM)\nSeptember 26, 2023\nPick Up Location\n1402 Williams Dr, Georgetown, TX 78628, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\n19 Crimes Cabernet Sauvignon 750ml Bottle (14% ABV) (Specialty Wine)\n$13.79\nSubtotal\n$13.79\nSubtotal Tax\n$1.76\nCommission (16%)\n-$2.21\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$13.34\nAssociated Transactions (1)\nTransaction #8072672500 - Delivery\n$13.34',
 'Order: 42CEA1A6\nDelivered\nThe order was delivered at 11:25 PM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nChris G\nDasher\nNardeah\nPicked Up\n11:20 PM\n(Quoted: 10:53 PM)\nSeptember 26, 2023\nDropped Off\n11:25 PM\n(Quoted: 11:13 PM)\nSeptember 26, 2023\nPick Up Location\n1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nIce Breakers Mint Coolmint (1.5 oz) (Mint TX)\n$5.69\nSubtotal\n$5.69\nSubtotal Tax\n$0.73\nCommission (20%)\n-$1.14\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$5.28\nAssociated Transactions (1)\nTransaction #8072574397 - Delivery\n$5.28',
 'Order: 6AB38CB1\nDelivered\nThe order was delivered at 11:24 PM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nChris G\nDasher\nNardeah\nPicked Up\n11:20 PM\n(Quoted: 10:55 PM)\nSeptember 26, 2023\nDropped Off\n11:24 PM\n(Quoted: 11:16 PM)\nSeptember 26, 2023\nPick Up Location\n1301 N U.S. Hwy 281, Marble Falls, TX 78654, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n2\n×\nIce Breakers Mint Coolmint (1.5 oz) (Mint TX)\n$11.38\nSubtotal\n$11.38\nSubtotal Tax\n$1.45\nCommission (20%)\n-$2.28\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$10.55\nAssociated Transactions (1)\nTransaction #8072573625 - Delivery\n$10.55',
 'Order: 4379B60E\nDelivered\nThe order was delivered at 10:52 PM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nDominic C\nDasher\nAdrian\nPicked Up\n10:35 PM\n(Quoted: 10:42 PM)\nSeptember 26, 2023\nDropped Off\n10:52 PM\n(Quoted: 10:57 PM)\nSeptember 26, 2023\nPick Up Location\n7045 W Del Mar Blvd, Laredo, TX 78041, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nSmirnoff Ice Original Malt Beverage Bottle (11.2 oz x 6 ct) (Flavored Malt Beverages Laredo)\n$14.99\nSubtotal\n$14.99\nSubtotal Tax\n$1.91\nCommission (16%)\n-$2.40\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$14.50\nAssociated Transactions (1)\nTransaction #8072378844 - Delivery\n$14.50',
 'Order: 0A150B93\nDelivered\nThe order was delivered at 10:34 PM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nAlyssa M\nDasher\nYunier\nPicked Up\n10:14 PM\n(Quoted: 10:20 PM)\nSeptember 26, 2023\nDropped Off\n10:34 PM\n(Quoted: 10:53 PM)\nSeptember 26, 2023\nPick Up Location\n2503 TX-349, Midland, TX 79706, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nCheetos Crunchy Jalapeno (8.5 oz) (Chips & Popcorn TX)\n$3.19\nSubtotal\n$3.19\nSubtotal Tax\n$0.00\nCommission (16%)\n-$0.51\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$2.68\nAssociated Transactions (1)\nTransaction #8072265117 - Delivery\n$2.68',
 'Order: E0EB8837\nDelivered\nThe order was delivered at 8:18 PM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nTyler P\nDasher\nEulises\nPicked Up\n7:58 PM\n(Quoted: 8:03 PM)\nSeptember 26, 2023\nDropped Off\n8:18 PM\n(Quoted: 8:29 PM)\nSeptember 26, 2023\nPick Up Location\n5004 Wesley St, Greenville, TX 75402, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n3\n×\nSteel Reserve Original Can (25 oz) (Flavored Malt Beverages NT)\n$13.17\nSubtotal\n$13.17\nSubtotal Tax\n$1.68\nCommission (16%)\n-$2.11\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$12.74\nAssociated Transactions (1)\nTransaction #8071200562 - Delivery\n$12.74',
 'Order: 77896463\nCustomer Picked Up\nThe customer picked up the order at 8:05 PM on September 26, 2023. \nGet help\nDelivery Details\nCustomer\nDavid P\nPicked Up\n8:05 PM\n(Quoted: 7:56 PM)\nSeptember 26, 2023\nPick Up Location\n7045 W Del Mar Blvd, Laredo, TX 78041, USA\nChannel\nDoorDash\nFulfillment\nConsumer Pickup\nOrder Details\n1\n×\nNatural Light Beer Can (12 oz x 15 ct) (Domestic Beer Laredo)\n$13.99\nSubtotal\n$13.99\nSubtotal Tax\n$1.78\nCommission (9%)\n-$1.26\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$14.51\nAssociated Transactions (1)\nTransaction #8071268296 - Pickup\n$14.51',
 'Order: B48F2460\nDelivered\nThe order was delivered at 7:46 PM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nDollymae J\nDasher\nAngela\nPicked Up\n7:26 PM\n(Quoted: 6:33 PM)\nSeptember 26, 2023\nDropped Off\n7:46 PM\n(Quoted: 7:32 PM)\nSeptember 26, 2023\nPick Up Location\n14620 Ranch Rd 12, Wimberley, TX 78676, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n3\n×\nCayman Jack Margarita Can (19 oz) (Flavored Malt Beverages CT)\n$17.07\nSubtotal\n$17.07\nSubtotal Tax\n$2.18\nCommission (23%)\n-$3.93\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$15.32\nAssociated Transactions (1)\nTransaction #8070880369 - Delivery\n$15.32',
 'Order: 78588F69\nDelivered\nThe order was delivered at 6:35 PM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nSteven L\nDasher\nFrancisco\nPicked Up\n6:27 PM\n(Quoted: 6:35 PM)\nSeptember 26, 2023\nDropped Off\n6:35 PM\n(Quoted: 6:54 PM)\nSeptember 26, 2023\nPick Up Location\n5004 Wesley St, Greenville, TX 75402, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n2\n×\nIce Large (Fountain, Frozen, & Tea)\n$14.98\nSubtotal\n$14.98\nSubtotal Tax\n$1.91\nCommission (16%)\n-$2.40\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$14.49\nAssociated Transactions (1)\nTransaction #8070265277 - Delivery\n$14.49',
 'Order: 5C857863\nCancelled - Not Paid\nThe order was cancelled on September 26, 2023 at 5:59 PM. You were not paid because you did not confirm the order before the order was cancelled. Learn More\nGet help\nDelivery Details\nCustomer\nDollymae J\nDasher\nTo be assigned\nEstimated Pick Up\n9:05 PM\nSeptember 26, 2023\nEstimated Drop Off\n9:30 PM\nSeptember 26, 2023\nPick Up Location\n14620 Ranch Rd 12, Wimberley, TX 78676, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n3\n×\nCayman Jack Margarita Can (19 oz) (Flavored Malt Beverages CT)\n$17.07\nSubtotal\n$17.07\nSubtotal Tax\n$2.18\nCommission (23%)\n-$3.93\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$0.00',
 'Order: CA2588A4\nDelivered\nThe order was delivered at 4:02 PM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nTyler P\nDasher\nJesse\nPicked Up\n3:44 PM\n(Quoted: 3:44 PM)\nSeptember 26, 2023\nDropped Off\n4:02 PM\n(Quoted: 4:09 PM)\nSeptember 26, 2023\nPick Up Location\n5004 Wesley St, Greenville, TX 75402, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n3\n×\nSteel Reserve Original Can (25 oz) (Flavored Malt Beverages NT)\n$13.17\nSubtotal\n$13.17\nSubtotal Tax\n$1.68\nCommission (16%)\n-$2.11\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$12.74\nAssociated Transactions (1)\nTransaction #8068891159 - Delivery\n$12.74',
 'Order: B6233BDE\nDelivered\nThe order was delivered at 11:55 AM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nJoshua C\nDasher\nOswaldo\nPicked Up\n11:41 AM\n(Quoted: 11:43 AM)\nSeptember 26, 2023\nDropped Off\n11:55 AM\n(Quoted: 12:10 PM)\nSeptember 26, 2023\nPick Up Location\n1001 W Van Alstyne Pkwy, Van Alstyne, TX 75495, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n2\n×\nGatorade Lime Cucumber Bottle (28 oz) (Sports Drinks TX)\n$9.58\n1\n×\nArizona Green Tea Bottle (34oz) (Tea TX)\n$4.39\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nCoke\n$0.00\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nMountain Dew\n$0.00\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nGatorade Lemon-Lime\n$0.00\nSubtotal\n$13.97\nSubtotal Tax\n$1.22\nCommission (23%)\n-$3.21\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$11.98\nAssociated Transactions (1)\nTransaction #8067195196 - Delivery\n$11.98',
 'Order: 8903B15F\nDelivered\nThe order was delivered at 11:41 AM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nPaul H\nDasher\nBridgette\nPicked Up\n11:35 AM\n(Quoted: 11:35 AM)\nSeptember 26, 2023\nDropped Off\n11:41 AM\n(Quoted: 11:50 AM)\nSeptember 26, 2023\nPick Up Location\n800 W Interstate 20, Big Spring, TX 79720, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nFritos Scoops Corn Chips (9.25 oz) (Chips & Popcorn TX)\n$6.49\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nDiet Dr Pepper\n$0.00\n1\n×\nFountain Drink (Fountain, Frozen, & Tea)\nDrink Selection:\nCoke\n$0.00\nSubtotal\n$6.49\nSubtotal Tax\n$0.00\nCommission (16%)\n-$1.04\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$5.45\nAssociated Transactions (1)\nTransaction #8067145629 - Delivery\n$5.45',
 'Order: 9EAC6D9E\nCancelled - Not Paid\nThe order was cancelled on September 26, 2023 at 10:39 AM. You were not paid because you did not confirm the order before the order was cancelled. Learn More\nGet help\nDelivery Details\nCustomer\nGillian T\nDasher\nTo be assigned\nEstimated Pick Up\n11:04 AM\nSeptember 26, 2023\nEstimated Drop Off\n11:32 AM\nSeptember 26, 2023\nPick Up Location\n3071 University Dr E, Bryan, TX 77802, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n2\n×\nBacon Egg Cheese Biscuit (Breakfast Sandwiches & Bowls)\n$9.78\n1\n×\nMonster Java Loca Mocha Can (15oz) (Coffee TX)\n$5.69\n3\n×\nPotato Egg Cheese Taco (Breakfast Tacos & Quesadillas)\nSalsa:\nSalsa Verde\n$14.07\nSubtotal\n$29.54\nSubtotal Tax\n$3.77\nCommission (23%)\n-$6.79\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$0.00',
 'Order: ABDD1702\nDelivered\nThe order was delivered at 12:00 AM on September 26, 2023. \nRate Dasher\nGet help\nDelivery Details\nCustomer\nDustin G\nDasher\nEverlena\nPicked Up\n11:46 PM\n(Quoted: 11:38 PM)\nSeptember 25, 2023\nDropped Off\n12:00 AM\n(Quoted: 11:55 PM)\nSeptember 26, 2023\nPick Up Location\n5004 Wesley St, Greenville, TX 75402, USA\nChannel\nDoorDash\nFulfillment\nDoorDash Delivery\nOrder Details\n1\n×\nCoors Original Beer Bottle (12oz x 6ct) (Domestic Beer NT)\n$14.99\nSubtotal\n$14.99\nSubtotal Tax\n$1.91\nCommission (23%)\n-$3.45\nTotal Customer Refund\n-$0.00\nEstimated Payout\n$13.45\nAssociated Transactions (1)\nTransaction #8065637009 - Delivery\n$13.45']

In [23]:
class Main:
    
    # ---------------------------------- Instance attributes ----------------------------------
    def __init__(self, headless=False):
        self.base_driver = BaseDriver(headless=headless)
        self.orders_page_driver = OrdersPageDriver(self.base_driver)
        self.today = datetime.today().strftime('%m.%d.%y')
        self.excel_file_name = f'DD {self.today}.xlsx'
        self.excel_output_file_path = f"/Users/ekim/workspace/txb/mock/g-drive/imports/ir/Door Dash/DD Daily Order Details/{self.excel_file_name}"

        # G:\Imports\IR\Door Dash\DD Daily Order Details # prod
    # ---------------------------------- Instance attributes ----------------------------------

    
    def switch_to_history_tab(self):
        switched_to_history_tab = self.orders_page_driver.switch_to_history_tab()
        if not switched_to_history_tab:
            logging.error('Could not orders_page_driver.switch_to_history_tab')
            return False
        else:
            logging.info('Successfully orders_page_driver.switch_to_history_tab')
            return True

    def set_date_filter_to_yesterday(self):
        date_filter_set_to_yesterday = self.orders_page_driver.set_date_filter_to_yesterday()
        if not date_filter_set_to_yesterday:
            logging.error('Could not orders_page_driver.set_date_filter_to_yesterday')
            return False
        else:
            logging.info('Successfully orders_page_driver.set_date_filter_to_yesterday')
            return True


    def switch_to_history_tab_and_set_date_filter_to_yesterday(self):
        try:

            switched_to_history_tab = self.switch_to_history_tab()
            if not switched_to_history_tab:
                logging.error('Could not switch_to_history_tab in switch_to_history_tab_and_set_date_filter_to_yesterday')
                return False, False

            date_filter_set_to_yesterday = self.set_date_filter_to_yesterday()
            if not date_filter_set_to_yesterday:
                logging.error('Could not set_date_filter_to_yesterday in switch_to_history_tab_and_set_date_filter_to_yesterday')
                return True, False

            if switched_to_history_tab and date_filter_set_to_yesterday:
                logging.info(f'Successfully switched_to_history_tab and date_filter_set_to_yesterday')
                return True, True

        except Exception as e:
            logging.exception(f'An error occurred attempting to switch_to_history_tab_and_set_date_filter_to_yesterday {e}')
            return False, False
            
    def get_sheet_name(self, order_df):
        try:
            store_num = order_df[0]['Store Number']
            sheet_name = f'#{store_num}'
            
            if sheet_name == '#NaN':
                logging.warning(f"Order: '{order_df[0]['Order']}'\'s Store Number could not be properly matched using Pick Up Location: '{order_df[0]['Pick Up Location']}'. Sheet name has been set to NaN.")
            
            return sheet_name, True
        except Exception as e:
            logging.exception(f"An error occurred while getting sheet name: {e}")
            return None, False
        
    def get_orders(self):
        try:
            orders = self.orders_page_driver.get_orders()
            if not orders:
                logging.error('Could not scrape orders data')
                return None
            else:
                return orders
        except Exception as e:
            logging.exception(f'An error occurred: {e}')
            return None
        
    # 
    def get_excel_output(self, orders_dfs):
        try:
            with pd.ExcelWriter(self.excel_output_file_path, engine='xlsxwriter') as writer:
                for idx, order_df in enumerate(orders_dfs):
                    sheet_name, success = self.get_sheet_name(order_df)
                    if not success:
                        return False

                    order_df.to_excel(writer, sheet_name, header=False, index=True)
                    
                    formatter = ExcelFormatter(writer, sheet_name, order_df)
                    formatter.apply_sheet_formats()

            return True
        except Exception as e:
            logging.exception(f"An error occurred while generating Excel output: {e}")
            return False

    def run_main(self):
        self.setup()
        if not self.switch_to_history_tab_and_set_date_filter_to_yesterday():
            logging.error("Failed to initialize DoorDash Bot")
            return

        # raw_orders = 

        orders_with_store_nums = self.merge_data(raw_orders)
        orders_dfs = self.convert_to_dataframes(orders_with_store_nums)

        self.export_to_excel(orders_dfs)
        logging.info(f'Orders Spreadsheet has been saved to: {self.excel_output_file_path}')

    def setup(self):
        setup_logger()

    def get_raw_orders(self):
        return self.get_orders()

    def merge_data(self, raw_orders):
        orders_json = get_prettified_and_mapped_orders(raw_orders)
        self.output_json(orders_json, 'orders_json.csv', 'Writing orders_json stdout...')
        
        dm = DataMerger(orders_json)
        return dm.add_store_numbers_to_orders()

    def output_json(self, json_str, filepath, log_message):
        output_filepath = os.path.join('/Users/ekim/workspace/personal/dd-bot/dev/build', filepath)
        json_str_to_file(json_str=json_str, output_filepath=output_filepath, log_message=log_message)

    def convert_to_dataframes(self, orders_with_store_nums):
        orders_json_with_store_nums = get_prettified_and_mapped_orders(orders_with_store_nums, with_store_nums=True)
        self.output_json(orders_json_with_store_nums, 'orders_json_with_store_nums.csv', 'Writing orders_json with store_num to stdout...')
        
        return convert_flattened_orders_to_df(orders_with_store_nums)

    def export_to_excel(self, orders_dfs):
        self.get_excel_output(orders_dfs)


# if __name__ == '__main__':
#     parser = argparse.ArgumentParser(description='DoorDash Bot V1')
#     parser.add_argument('--headless', required=False, action='store_true', help='Run in headless mode')
#     args = parser.parse_args()
# 
#     md = Main()
#     md.run_main(headless=args.headless)

md = Main()
md.run_main()


2023-10-05 12:50:48,028 - INFO - Initializing BaseDriver...
2023-10-05 12:50:48,567 - INFO - Using operating system: "Darwin".
Constructing chromedriver instance using executable_path: "/opt/homebrew/bin/chromedriver"
2023-10-05 12:50:48,663 - INFO - Successfully orders_page.switch_to_history_tab
2023-10-05 12:50:48,663 - INFO - Successfully orders_page_driver.switch_to_history_tab
2023-10-05 12:50:48,710 - INFO - Waiting for date filter button click to load on DOM ....
2023-10-05 12:50:53,816 - INFO - Waiting for yesterday date filter selection click to load on DOM...
2023-10-05 12:50:58,825 - INFO - Successfully set date filter to yesterday
2023-10-05 12:50:58,826 - INFO - Successfully orders_page.set_date_filter_to_yesterday
2023-10-05 12:50:58,829 - INFO - Successfully orders_page_driver.set_date_filter_to_yesterday
2023-10-05 12:50:58,830 - INFO - Successfully switched_to_history_tab and date_filter_set_to_yesterday
2023-10-05 12:50:58,833 - INFO - Prettifying data...
2023-10-05 1